In [2]:
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt
%matplotlib inline

$K_{3x3}=\begin{bmatrix}
c & cs     & X_h\\
  & c(1+m) & Y_h\\
  &        & 1\\
\end{bmatrix}=
\begin{bmatrix}
1 & s   & X_h \\
  & 1+m & Y_h \\
  &      & 1
\end{bmatrix}*\begin{bmatrix}
c &    &  \\
  & c  &  \\
  &    & 1
\end{bmatrix}
$

$R_{3x4}=
\begin{bmatrix}
\overrightarrow{r1} & \overrightarrow{r2} & \overrightarrow{r3} & \overrightarrow{t}
\end{bmatrix}
$


**Hints**:
- $\overrightarrow{r1} , \overrightarrow{r2} , \overrightarrow{r3}$ 分别是x,y,z轴在camera坐标系统中的方向。
- $\overrightarrow{t}是 O 点在camera坐标系统中的位置。$

In [3]:
def KMatrix(c,m,s,xh,yh):
    M=[
        [c,c*s,xh],
        [0,c*(1+m),yh],
        [0,0,1]
    ]
    return np.array(M)
# 返回3x4的矩阵 [R,-RX0]
def RMatrix(xdegre,ydegress,zdegress,X0):
    def Rx(d):
        c,s=np.cos(d/180*np.pi),np.sin(d/180*np.pi)
        M=[
            [1,0, 0],
            [0,c,-s],
            [0,s, c],
        ]
        return np.array(M)
    def Ry(d):
        c,s=np.cos(d/180*np.pi),np.sin(d/180*np.pi)
        M=[
           [ c,0,s],
           [ 0,1,0],
           [-s,0,c],
        ]
        return np.array(M)
    def Rz(d):
        c,s=np.cos(d/180*np.pi),np.sin(d/180*np.pi)
        M=[
            [c,-s,0],
            [s, c,0],
            [0, 0,1],
        ]
        return np.array(M)
    R=Rx(xdegre)@Ry(ydegress)@Rz(zdegress)
    h=-R.T@np.array(X0)
    return np.concatenate([R.T,-h[:,np.newaxis]],axis=1)

In [4]:
K=KMatrix(c=-33,m=0.3,s=0.87,xh=33,yh=22)
R34=RMatrix(xdegre=30,ydegress=-20,zdegress=55,X0=[3,2,1])

np.testing.assert_array_almost_equal(R34[:,0].T@R34[:,1],0)
np.testing.assert_array_almost_equal(R34[:,0].T@R34[:,2],0)
np.testing.assert_array_almost_equal(R34[:,1].T@R34[:,2],0)
np.testing.assert_array_almost_equal(np.linalg.norm(R34[:,0]),1)
np.testing.assert_array_almost_equal(np.linalg.norm(R34[:,1]),1)
np.testing.assert_array_almost_equal(np.linalg.norm(R34[:,2]),1)

print('K=',K)
print('R=\n',R34)

K= [[-33.   -28.71  33.  ]
 [  0.   -42.9   22.  ]
 [  0.     0.     1.  ]]
R=
 [[ 0.53898554  0.61131913  0.57946829  3.41906319]
 [-0.76975113  0.63681501  0.04415691 -0.99146645]
 [-0.34202014 -0.46984631  0.81379768 -1.15195537]]


### 坐标映射

$\overrightarrow{X_w}=\begin{bmatrix} x \\y \\z \\1 \end{bmatrix}$ ,
$\overrightarrow{x_s}=\begin{bmatrix} x \\y  \\1 \end{bmatrix}$,以下公式把2个坐标联系起来:

$$\overrightarrow{x_s}=\lambda K_{3x3}R_{3x4}\overrightarrow{X_w}$$

**Hints**
- $\overrightarrow{x_s},\overrightarrow{X_w}都是齐次坐标，最后一个维度都是1$。

### DLT

由坐标映射可知以下关系：

$$\overrightarrow{x^{(i)}_s}= P\overrightarrow{X^{(i)}_w}$$
**Hints**

$$
M^{(i)}=
\begin{bmatrix}
    X^T_w && 0        && -x_sX^T_w \\
    0     &&  -X^T_w  && -y_sX^T_w \\
\end{bmatrix} ^{(i)} \overrightarrow{P}=0
$$
- $\overrightarrow{x^{(i)}_s},\overrightarrow{X^{(i)}_w}$是已知数据，估算P。
- 12(11)个参数，需要6的采样点的对应关系。

In [5]:
# X:(N,4), (x,y,z,1) in world coordinate or (N,3), (x,y,1) in checkboard
# x:(N,3), (x,y,1) in sensor coordinate
# return: estimate parameter H=KR
def DLT(X,x):
    np.testing.assert_equal(len(X),len(x))
    G=[]
    N=len(X)
    
    for i in range(N):
        Xi=X[i]
        xi,yi,_=x[i]
        ai=np.concatenate([Xi,np.zeros(len(Xi)),-xi*Xi])
        bi=np.concatenate([np.zeros(len(Xi)),Xi,-yi*Xi])
        G.append(ai)
        G.append(bi)
    G=np.array(G) 
    
    #G.shape=(2N,9)
    U,S,VT=np.linalg.svd(G)
    H=VT.T[:,-1]
    H.shape=(3,len(Xi))
    return H

In [6]:
##生成 3D坐标X 与2D 坐标x 的对应 关系
# X：（N,4）,齐次坐标系
# x:  (N,3),齐次坐标系

P=K@R34

n=6
X=np.random.rand(n,4)
X[:,3]=1  #正则

px=X@P.T # 投影
x=px/px[:,2:] #正则

In [7]:
##估算参数
P_es=DLT(X,x)
np.testing.assert_almost_equal(P/P[-1,-1],P_es/P_es[-1,-1])

### QR 还原K,R

$
 P=\lambda \begin{bmatrix}
 KR_{3x3} & Kt
 \end{bmatrix}=
  \begin{bmatrix}
  H & h
 \end{bmatrix}
$

$
 H^{-1}=R^{-1}K^{-1}=qr(H)
$

结论：$ R=q^T,K=r^{-1},t=(\lambda K)^{-1}h$,

$
F=\begin{bmatrix}
 -1 &  0 & 0 \\
  0 & -1 & 0 \\
  0 &  0 & 1
 \end{bmatrix}
$

**Hints**:

- K*R=(KF)*(FR),所以如果$K_{00}>0$,可以用这个方法修正。
- 有如下约束：$K_{33}=1,R^TR=I$,所以经过QR分解后,未知系数$\lambda$可以被确定下来。

In [71]:
#还原K,R
# print(Rz(180))
perm=np.array([[-1,0,0],[0,-1,0],[0,0,1]])

H=P_es[:3,:3]
h=P_es[:,3]


q,r=np.linalg.qr(np.linalg.inv(H))
R_es=q.T
K_es=np.linalg.inv(r)
K_es=K_es/K_es[-1][-1]

# 修正
if K_es[0,0]>0:
    K_es=K_es@perm
    R_es=perm@R_es
np.testing.assert_almost_equal(R_es,R34[:3,:3])
np.testing.assert_almost_equal(K_es,K)

# t 是 0的位置
labda=(H/(K_es@R_es))[0,0]
t=np.linalg.solve(labda*K_es,h)
np.testing.assert_almost_equal(t,R34[:,3])


### zhang's method

In [8]:
def Vij(h,i,j):
    return np.array([
        h[0,i]*h[0,j],
        h[1,i]*h[0,j]+h[1,j]*h[0,i],
        h[2,i]*h[0,j]+h[2,j]*h[0,i],
        h[1,i]*h[1,j],
        h[1,i]*h[2,j]+h[1,j]*h[2,i],
        h[2,i]*h[2,j],
    ])